In [2]:

download_language = {"English": "en",
"Spanish": "es",
"French": "fr",
"Italian": "it",
"German": "de",
"Portuguese": "pt",
"Dutch": "nl",
"Swedish": "sv",
"Norwegian": "no",
"Danish": "da",
"Finnish": "fi",
"Latin": "la",
"Romanian": "ro",
"Catalan": "ca",
"Hungarian": "hu",
"Polish": "pl",
"Czech": "cs",
"Slovak": "sk",
"Slovenian": "sl",
"Croatian": "hr"}

print(len(download_language))

20


In [3]:
# for d in datasets_list:
#     if "wikimedia" in d:
#         print(d)
#         break

In [2]:
import yaml

with open("../data/huggingface_data_README.yml", "r") as stream:
    try:
        dataset_readme = yaml.safe_load(stream)
    except yaml.YAMLError as exc:
        print(exc)


In [11]:
from langcodes import Language

def get_language_name(iso_code):
    try:
        lang = Language.get(iso_code)
        return lang.display_name().title()
    except ValueError:
        return None

In [13]:
 

configs = dataset_readme['configs']
conf_list = {}
for conf in configs:
    code = conf['config_name'].split('.')[-1]
    lang = get_language_name(code)
    if lang is not None:
        conf_list[lang]= code 
print(len(conf_list))
print(conf_list)

320
{'Abkhazian': 'ab', 'Achinese': 'ace', 'Adyghe': 'ady', 'Afrikaans': 'af', 'Tosk Albanian': 'als', 'Southern Altai': 'alt', 'Amharic': 'am', 'Amis': 'ami', 'Aragonese': 'an', 'Old English': 'ang', 'Angika': 'anp', 'Arabic': 'ar', 'Aramaic': 'arc', 'Moroccan Arabic': 'ary', 'Egyptian Arabic': 'arz', 'Assamese': 'as', 'Asturian': 'ast', 'Atikamekw': 'atj', 'Avaric': 'av', 'Kotava': 'avk', 'Awadhi': 'awa', 'Aymara': 'ay', 'Azerbaijani': 'az', 'South Azerbaijani': 'azb', 'Bashkir': 'ba', 'Balinese': 'ban', 'Bavarian': 'bar', 'Baltic Languages': 'bat-smg', 'Central Bikol': 'bcl', 'Belarusian': 'be-x-old', 'Bulgarian': 'bg', 'Bihari Languages': 'bh', 'Bislama': 'bi', 'Banjar': 'bjn', "Pa'O Karen": 'blk', 'Bambara': 'bm', 'Bangla': 'bn', 'Tibetan': 'bo', 'Bishnupriya': 'bpy', 'Breton': 'br', 'Bosnian': 'bs', 'Buginese': 'bug', 'Russia Buriat': 'bxr', 'Catalan': 'ca', 'Chavacano': 'cbk-zam', 'Min Dong Chinese': 'cdo', 'Chechen': 'ce', 'Cebuano': 'ceb', 'Chamorro': 'ch', 'Cherokee': 'chr', 

In [3]:
# downloads= set(['en', 'fr'])
downloads= set(download_language.values())

configs = dataset_readme['configs']
conf_list = []
for conf in configs:
    if conf['config_name'].split('.')[-1] in downloads:
        cnf = conf['config_name']
        conf_list.append(cnf)

In [5]:
len(conf_list)

20

In [8]:
import re

def get_parquet_file_list(text:str):
    pattern = r'download href="/datasets/wikimedia/wikipedia/resolve/main/.*\.parquet\?download=true">\d* MB'
    substrings = re.findall(pattern, text)
    return substrings


In [9]:
import requests

def get_one_parqet_file_per_conf(cnf):
    base_url = 'https://huggingface.co/datasets/wikimedia/wikipedia/tree/main/'
    url = base_url  + cnf
    resp = requests.get(url)
    text = resp.content.decode('utf-8')
    parquet_files = get_parquet_file_list(text)
    
    urls_parquet = []
    for f in parquet_files:
        f = f.replace('download href="', '').replace("MB", '')
    
        url_p = f.split('">')[0]
        size_p = f.split('">')[-1]
        urls_parquet.append([url_p.strip(), size_p.strip()])
    
    
    urls_parquet = sorted(urls_parquet, key=lambda u: u[-1])

    min_size = 150 # MB, try to get the small file, but greater than this if possible
    if len(urls_parquet) ==1: 
        return urls_parquet[0]
    else:
        result = None
        for u in urls_parquet:
            result = u
            if int(result[-1]) > min_size:
                return result
        return result
            
    

In [10]:

parquet_file_paths = []
for conf in conf_list:
     parquet_file_paths.append((get_one_parqet_file_per_conf(conf), conf.split('.')[-1]))

parquet_file_paths  

[(['/datasets/wikimedia/wikipedia/resolve/main/20231101.ca/train-00001-of-00004.parquet?download=true',
   '209'],
  'ca'),
 (['/datasets/wikimedia/wikipedia/resolve/main/20231101.cs/train-00002-of-00004.parquet?download=true',
   '195'],
  'cs'),
 (['/datasets/wikimedia/wikipedia/resolve/main/20231101.da/train-00000-of-00002.parquet?download=true',
   '203'],
  'da'),
 (['/datasets/wikimedia/wikipedia/resolve/main/20231101.de/train-00014-of-00020.parquet?download=true',
   '220'],
  'de'),
 (['/datasets/wikimedia/wikipedia/resolve/main/20231101.en/train-00028-of-00041.parquet?download=true',
   '188'],
  'en'),
 (['/datasets/wikimedia/wikipedia/resolve/main/20231101.es/train-00010-of-00013.parquet?download=true',
   '167'],
  'es'),
 (['/datasets/wikimedia/wikipedia/resolve/main/20231101.fi/train-00002-of-00003.parquet?download=true',
   '171'],
  'fi'),
 (['/datasets/wikimedia/wikipedia/resolve/main/20231101.fr/train-00011-of-00017.parquet?download=true',
   '169'],
  'fr'),
 (['/dat

In [12]:
import os

def download_file(path, local_filename):
    base_url = "https://huggingface.co"
    url = base_url + path
    response = requests.get(url, stream=True)
    
    with open(local_filename, 'wb') as file:
        for chunk in response.iter_content(chunk_size=128):
            file.write(chunk)


for pths in parquet_file_paths:
    if pths[0] is None:
        continue

    
    file_path  = pths[0][0]
    lang = pths[1]
    
    # print(file_path)
    file_save_as = '../data/parquet/' + lang+"."+file_path.split('?')[0].split('/')[-1]
    
    # print(file_save_as)

    if not os.path.exists(file_save_as):
        print(f'downloading {file_path} to {file_save_as}')
        download_file(file_path, file_save_as)
    else: 
        print(f'{file_save_as} is existing.')



../data/parquet/ca.train-00001-of-00004.parquet is existing.
../data/parquet/cs.train-00002-of-00004.parquet is existing.
../data/parquet/da.train-00000-of-00002.parquet is existing.
../data/parquet/de.train-00014-of-00020.parquet is existing.
../data/parquet/en.train-00028-of-00041.parquet is existing.
../data/parquet/es.train-00010-of-00013.parquet is existing.
../data/parquet/fi.train-00002-of-00003.parquet is existing.
../data/parquet/fr.train-00011-of-00017.parquet is existing.
../data/parquet/hr.train-00000-of-00001.parquet is existing.
../data/parquet/hu.train-00003-of-00004.parquet is existing.
../data/parquet/it.train-00007-of-00010.parquet is existing.
downloading /datasets/wikimedia/wikipedia/resolve/main/20231101.nl/train-00001-of-00006.parquet?download=true to ../data/parquet/nl.train-00001-of-00006.parquet
downloading /datasets/wikimedia/wikipedia/resolve/main/20231101.no/train-00001-of-00003.parquet?download=true to ../data/parquet/no.train-00001-of-00003.parquet
downloa